In [1]:
import torch 
import torch.nn as nn
import pytorch_lightning as pl
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset, random_split


In [2]:

# Embedding vector --> From LLM Encoder & Graph encoder || MHC & peptide 
# Siam model ( MHC, Peptide)-> 1 if binds else 0 
# input_dim = 1280

In [3]:
class EmbeddingDataset(Dataset):
    
    def __init__(self, hla_embedding, peptide_embedding) -> None:
        super(EmbeddingDataset).__init__()

        self.hla_embedding = hla_embedding
        self.peptide_embedding = peptide_embedding

    def __len__(self):
        assert len(self.hla_embedding) == len(self.peptide_embedding)
        return len(self.hla_embedding)
    
    def __getitem__(self, idx):        
        return self.hla_embedding[idx], self.peptide_embedding[idx]



class EmbeddingDataModule(pl.LightningDataModule):

    def __init__(self, hla, peptide ,batch_size):
        super(EmbeddingDataModule).__init__()
        self.hla = hla     
        self.peptide = peptide 
        self.batch_size = batch_size


    def setup(self, stage=None):
        full_dataset = EmbeddingDataset(self.hla,self.peptide)
        # datasplit = random_split(full_dataset, [int(len(full_dataset) * 0.8), len(full_dataset) - int(len(full_dataset) * 0.8)])
        # self.train_ids,self.val_ids = datasplit[0].dataset.identifiers,datasplit[1].dataset.identifiers
        self.train, self.val = random_split(full_dataset, [int(len(full_dataset) * 0.8), len(full_dataset) - int(len(full_dataset) * 0.8)])
        # self.train, self.val = datasplit[0].dataset.data,datasplit[1].dataset.data

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=4)





In [4]:


def contrastive_loss(vec1, vec2, score_assoc):

    margin = 1.5
    euclidean_distance  = (vec1 - vec2).pow(2).sum(1).sqrt()

    return 1/2*( score_assoc * euclidean_distance.pow(2) + (1-score_assoc)*max(0,margin-euclidean_distance)**2 )


class SiameseModel(nn.Module):
    def __init__(self,input_dim ) -> None:
        super(SiameseModel,self).__init__()
        self.input_dim = input_dim

        self.featurizer = nn.Sequential(
                nn.Conv1d(self.input_dim, 512, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.Conv1d(512, 256, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.AdaptiveMaxPool1d(1) ,
                nn.Flatten() 

            )
        
        self.conv2d_block = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), stride=1, padding=1), 
            nn.ReLU(),
            nn.AdaptiveMaxPool2d(1),  # [batch_size, 64, 1, 1]
            nn.Flatten(),
        )

        self.scorecompute_block = nn.Sequential(
            
                nn.Linear(128,32),
                nn.ReLU(),
                nn.Linear(32,1),
                nn.Sigmoid(),
        )

    
    # def comb_vecs(self, vec1, vec2):

    #     vec1, vec2 = torch.reshape(vec1,(1,128)),torch.reshape(vec2,(1,128))
    #     combined = torch.cat((vec1, vec2), dim=0)

    #     return torch.reshape(combined, (1,1,256))

    def forward(self,x1,x2):
        feat_x1,feat_x2 = self.featurizer(x1), self.featurizer(x2)

        print(feat_x1)
        print(feat_x2)
        print(feat_x1.shape)
        print(feat_x2.shape)

        # combined_representation = self.comb_vecs(feat_x1,feat_x2)
        # print(combined_representation.shape)

        combined_representation =torch.reshape(feat_x1 - feat_x2,(1,128))

        # conv_out = self.conv2d_block(combined_representation)
        # print(conv_out)
        
        score = self.scorecompute_block(combined_representation)
        print(score.shape)

        return score



    def training_step(self,batch,):
        mha, peptide, association = batch 

        predicted_association  = self.forward(mha,peptide)

        train_loss = contrastive_loss(mha,peptide,association)
        return train_loss
        








In [5]:

test = torch.randn((100,1,1280))



In [6]:
model = SiameseModel(1)

score = model(test[0],test[1])

tensor([[0.3327],
        [0.2204],
        [0.2611],
        [0.1550],
        [0.1666],
        [0.2379],
        [0.2070],
        [0.0826],
        [0.2781],
        [0.3370],
        [0.0769],
        [0.2095],
        [0.2235],
        [0.3379],
        [0.2914],
        [0.3359],
        [0.1132],
        [0.4395],
        [0.1183],
        [0.3960],
        [0.4694],
        [0.1434],
        [0.0789],
        [0.1746],
        [0.4421],
        [0.4930],
        [0.3121],
        [0.2238],
        [0.4491],
        [0.2017],
        [0.1347],
        [0.1856],
        [0.3160],
        [0.1928],
        [0.2157],
        [0.1965],
        [0.2368],
        [0.1044],
        [0.2375],
        [0.2485],
        [0.2615],
        [0.1593],
        [0.1363],
        [0.4353],
        [0.1925],
        [0.1758],
        [0.1042],
        [0.2023],
        [0.3383],
        [0.1548],
        [0.0883],
        [0.2208],
        [0.1793],
        [0.5429],
        [0.2748],
        [0

In [7]:
score.detach().numpy()



array([[0.53125024]], dtype=float32)

In [8]:
test[0]

tensor([[ 1.1858, -0.1531, -0.2003,  ...,  1.5323,  1.2208,  1.0693]])

In [9]:
test[1]

tensor([[ 1.5792,  0.4030,  1.6063,  ..., -1.4488,  0.7400,  1.1089]])

In [10]:
(test[0] - test[1]).pow(2).sum(1).sqrt()

tensor([50.5452])